<a href="https://colab.research.google.com/github/kirwarobert/cnn/blob/main/CHATBOT_USING_STREAMLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install streamlit pyngrok
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 1s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [29]:
%%writefile customer_care_chatbot.py
import streamlit as st
import random
import time
from datetime import datetime

# Set page config
st.set_page_config(page_title="Customer Care Chatbot", page_icon="🤖")

# Initialize session state for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Initialize session state for user info
if "user_info" not in st.session_state:
    st.session_state.user_info = {
        "name": None,
        "email": None,
        "issue": None
    }

# Common customer service responses
RESPONSES = {
    "greeting": [
        "Hello! Welcome to our customer support. How can I assist you today?",
        "Hi there! How can I help you today?",
        "Welcome! What can I do for you today?"
    ],
    "farewell": [
        "Thank you for contacting us. Have a great day!",
        "We're happy to help! Let us know if you need anything else.",
        "Your satisfaction is our priority. Goodbye!"
    ],
    "positive": [
        "Great to hear that! Is there anything else I can help with?",
        "That's wonderful! Let me know if you need anything else.",
        "I'm glad to hear that. How else can I assist you?"
    ],
    "negative": [
        "I'm sorry to hear that. Let me help resolve this for you.",
        "I apologize for the inconvenience. Let's get this sorted out.",
        "That doesn't sound good. Let me see what I can do to help."
    ],
    "escalate": [
        "Let me connect you with a human representative for further assistance.",
        "I'll transfer you to a specialist who can better handle your request.",
        "For this issue, you'll need to speak with one of our support agents."
    ],
    "confused": [
        "I'm not sure I understand. Could you please rephrase that?",
        "I want to make sure I help you correctly. Could you explain that differently?",
        "Let me try to understand better. Could you say that another way?"
    ]
}

# Common issues and solutions
ISSUE_DB = {
    "account": {
        "questions": ["account", "login", "password", "sign in"],
        "solutions": [
            "For account issues, please try resetting your password using the 'Forgot Password' link.",
            "Make sure you're using the correct email associated with your account.",
            "Account verification may be required. Check your email for a verification link."
        ]
    },
    "payment": {
        "questions": ["payment", "charge", "refund", "bill", "invoice"],
        "solutions": [
            "Payment processing usually takes 3-5 business days to reflect in your account.",
            "For refunds, please contact our billing department with your transaction ID.",
            "Double-check your payment method details and try the transaction again."
        ]
    },
    "delivery": {
        "questions": ["delivery", "ship", "track", "arrive", "shipping"],
        "solutions": [
            "Standard delivery takes 5-7 business days. Express shipping is 2-3 days.",
            "You can track your order using the tracking number provided in your confirmation email.",
            "Delivery times may vary based on your location and current order volume."
        ]
    },
    "technical": {
        "questions": ["error", "bug", "technical", "not working", "crash"],
        "solutions": [
            "Try clearing your browser cache and cookies, then restart the application.",
            "Make sure you're using the latest version of our app or software.",
            "This might be a temporary issue. Please try again after some time."
        ]
    }
}

def get_response(user_input):
    """Generate an appropriate response based on user input"""
    user_input = user_input.lower()

    # Check for greetings
    if any(word in user_input for word in ["hi", "hello", "hey"]):
        return random.choice(RESPONSES["greeting"])

    # Check for farewells
    if any(word in user_input for word in ["bye", "goodbye", "thanks"]):
        return random.choice(RESPONSES["farewell"])

    # Check for positive feedback
    if any(word in user_input for word in ["great", "good", "awesome", "thank"]):
        return random.choice(RESPONSES["positive"])

    # Check for negative feedback
    if any(word in user_input for word in ["bad", "terrible", "awful", "angry", "upset"]):
        return random.choice(RESPONSES["negative"])

    # Check for specific issues
    for issue, data in ISSUE_DB.items():
        if any(word in user_input for word in data["questions"]):
            solution = random.choice(data["solutions"])
            return f"Regarding your {issue} issue: {solution}"

    # If no specific match, check if user is providing information
    if "name is" in user_input or "i'm " in user_input:
        return "Thank you for providing that information. How can I assist you?"

    # Default response if nothing matches
    return random.choice(RESPONSES["confused"])

def collect_user_info():
    # Initialize session state with proper type
    if "user_info" not in st.session_state:
        st.session_state.user_info = ""  # Initialize as empty string

    # Ensure existing value is string type
    if not isinstance(st.session_state.get("user_info"), str):
        st.session_state.user_info = str(st.session_state.user_info)

    with st.form("user_info_form"):
        # Create text input with type-safe value
        name = st.text_input(
            "Your Name",
            value=str(st.session_state.user_info),  # Explicit string conversion
            key="user_info"
        )

        submitted = st.form_submit_button("Submit")

        if submitted:
            if name.strip():  # Check for non-empty string
                return True
            st.error("Please enter your name")
            return False

    return False

def main():
    """Main function to run the chatbot"""
    st.title("🤖 Customer Care Chatbot")
    st.write("Welcome to our 24/7 customer support service.")

    # Check if user info is collected
    if not all(st.session_state.user_info.values()):
        if collect_user_info():
            st.experimental_rerun()
        return

    # Display user info
    st.sidebar.subheader("Your Information")
    st.sidebar.write(f"Name: {st.session_state.user_info['name']}")
    st.sidebar.write(f"Email: {st.session_state.user_info['email']}")
    st.sidebar.write(f"Issue: {st.session_state.user_info['issue']}")

    # Display chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Accept user input
    if prompt := st.chat_input("How can I help you?"):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})

        # Display user message in chat message container
        with st.chat_message("user"):
            st.markdown(prompt)

        # Generate and display bot response
        with st.chat_message("assistant"):
            response = get_response(prompt)
            st.markdown(response)

        # Add bot response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()

Overwriting customer_care_chatbot.py


In [30]:
# Run the Streamlit app
!streamlit run customer_care_chatbot.py &>/dev/null&

In [35]:
!ngrok config add-authtoken 2vRplRczLuVgGShyebTSazSWA8O_4uZBG4UmeKkaEno7PANz4

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [36]:
from pyngrok import ngrok

# Set your authtoken

# Now you can connect as before
public_url = ngrok.connect(addr='localhost:8501')
public_url

<NgrokTunnel: "https://887e-34-121-230-56.ngrok-free.app" -> "http://localhost:8501">